In [19]:
import torch; torch.version.cuda

'12.1'

In [20]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset

max_seq_length = 2048

# 4bit pre quantized models we support - 4x faster downloading!
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
]

In [21]:
# Load Llama model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/tinyllama-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` attribute will be overwritten with the one you passed to `from_pretrained`.


In [22]:

# Do model patching and add fast LoRA weights
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = max_seq_length,
)

In [27]:
#@title Alpaca dataset preparation code
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["system_prompt"]
    inputs       = examples["question"]
    outputs      = examples["response"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output)
        texts.append(text)
    return { "text" : texts, }
pass


In [25]:
dataset = load_dataset("Open-Orca/OpenOrca", split = "train")

Generating train split: 4233923 examples [00:15, 269935.25 examples/s]
Map:   0%|          | 0/4233923 [00:00<?, ? examples/s]


KeyError: 'reponse'

In [28]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map: 100%|██████████| 4233923/4233923 [00:57<00:00, 73116.38 examples/s] 


In [30]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    packing=False,
    max_seq_length = max_seq_length,
    tokenizer = tokenizer,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 100,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        output_dir = "outputs",
        optim = "adamw_8bit",
        seed = 3407,
    ),
)

Map:  60%|██████    | 2561000/4233923 [10:09<06:34, 4239.63 examples/s]

In [17]:
trainer.train()

  2%|▏         | 1/60 [00:01<01:24,  1.43s/it]

{'loss': 1.1693, 'learning_rate': 5e-06, 'epoch': 0.0}


  3%|▎         | 2/60 [00:02<01:17,  1.34s/it]

{'loss': 1.2413, 'learning_rate': 1e-05, 'epoch': 0.0}


  5%|▌         | 3/60 [00:04<01:27,  1.53s/it]

...

 98%|█████████▊| 59/60 [01:20<00:01,  1.33s/it]

{'loss': 1.0384, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}


100%|██████████| 60/60 [01:22<00:00,  1.37s/it]

{'loss': 1.1878, 'learning_rate': 0.0, 'epoch': 0.01}
{'train_runtime': 82.0261, 'train_samples_per_second': 5.852, 'train_steps_per_second': 0.731, 'train_loss': 1.0796640833218893, 'epoch': 0.01}


TrainOutput(global_step=60, training_loss=1.0796640833218893, metrics={'train_runtime': 82.0261, 'train_samples_per_second': 5.852, 'train_steps_per_second': 0.731, 'train_loss': 1.0796640833218893, 'epoch': 0.01})

In [18]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
]*1, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\nThe fibonnaci sequence is a sequence of numbers that is continued from the first number to the second number. The first number in the sequence is 1, and the second number is 1. The next number in the sequence is 1, and the next number is 2. The next number in the sequence is 3, and the next number is 5. The next number in the sequence is 8, and the next number is 8. The next number in the sequence is 16, and the next number is 16. The next number in the sequence is 32, and the next']